In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# download the data from kaggle and unzip

! pip install -q kaggle

! mkdir ~/.kaggle

!cp /content/drive/MyDrive/cs231n/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

! unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

# install wandb if not present
! pip install -q wandb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists
challenges-in-representation-learning-facial-expression-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import io
from contextlib import redirect_stdout
from torchsummary import summary

wandb.init(project="ml_assignment_4", name="simple-cnn")

# Hyperparameters
config = {
    "epochs": 10,
    "batch_size": 64,
    "learning_rate": 1e-3,
    "image_size": 48,
    "num_classes": 7,
}
wandb.config.update(config)


# Dataset class
class FacialExpressionDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        pixels = np.array(self.df.iloc[idx]['pixels'].split(), dtype=np.uint8).reshape(48, 48)
        image = Image.fromarray(pixels)
        label = int(self.df.iloc[idx]['emotion']) if 'emotion' in self.df.columns else -1

        if self.transform:
            image = self.transform(image)

        return image, label

# Transforms
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
train_df = pd.read_csv(os.path.expanduser("/content/train.csv"))
train_data, val_data = train_test_split(train_df, test_size=0.1, stratify=train_df['emotion'], random_state=42)

train_dataset = FacialExpressionDataset(train_data, transform=transform)
val_dataset = FacialExpressionDataset(val_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

# Model
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 12 * 12, 256), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(num_classes=config["num_classes"]).to(device)

# log the model summary

f = io.StringIO()
with redirect_stdout(f):
    summary(model, input_size=(1, 48, 48))
model_summary_str = f.getvalue()

# Log to wandb as formatted HTML (nicely viewable in UI)
wandb.log({"model_summary": wandb.Html(f"<pre>{model_summary_str}</pre>")})

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

# Training loop
for epoch in range(config["epochs"]):
    model.train()
    running_loss, running_acc = 0.0, 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_acc += (preds == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = running_acc / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss, val_acc = 0.0, 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * images.size(0)
            val_acc += (preds == labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)

    # Log to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
    })

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

# Save and log model
torch.save(model.state_dict(), "model.pth")
wandb.save("model.pth")
wandb.finish()

Epoch 1: Train Acc=0.3534, Val Acc=0.4204
Epoch 2: Train Acc=0.4460, Val Acc=0.4789
Epoch 3: Train Acc=0.4897, Val Acc=0.5044
Epoch 4: Train Acc=0.5178, Val Acc=0.5110
Epoch 5: Train Acc=0.5444, Val Acc=0.5214
Epoch 6: Train Acc=0.5806, Val Acc=0.5249
Epoch 7: Train Acc=0.6047, Val Acc=0.5409
Epoch 8: Train Acc=0.6307, Val Acc=0.5388
Epoch 9: Train Acc=0.6568, Val Acc=0.5357
Epoch 10: Train Acc=0.6756, Val Acc=0.5423


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▅▅▆▆▇██
train_loss,█▆▅▅▄▃▃▂▂▁
val_accuracy,▁▄▆▆▇▇████
val_loss,█▅▃▂▁▁▁▁▂▃
epoch,10
train_accuracy,0.67563
train_loss,0.84784
val_accuracy,0.54232
val_loss,1.30192
